Import necessary packages

In [1]:
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from browserupproxy import Server
import psutil
from datetime import date, datetime
import time
from urllib.parse import urlparse

import os
import glob
import shutil

import pandas as pd

# Interact with WebPage - Selenium 
Detialed Guide is in <a href = 'https://selenium-python.readthedocs.io/getting-started.html'>selenium python</a>
1. Download a file on a click event using <a href = 'https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium'>selenium</a><br>
2. Install driver. a <b>Driver</b> is needed to interface with the chosen browser.For Chrome, use <a href = 'https://sites.google.com/a/chromium.org/chromedriver/downloads'>chromedriver</a> with right version. This driver is also needed to be installed and in the <i>PATH</i>. The port code: 9515<br>
3. Import Keys from <i>selenium.webdriver</i> module in order to get keys in the keyboard like RETURN, F1, ALT
4. Create a WebDriver instance
5. <i>driver.get</i> method will navigate to a page given by the URL. WebDriver will wait until the page has fully loaded before returning control to your script. 
6. WebDriver offers a number of ways to find elements using one of the <i>.find_element_by_*</i> methods. Then you can use <i>.send_keys('xxx')</i> to pass variables. Be sure to refer to the <b>page inspect</b> to help get the element information. For example, if you want to pass 'username' and 'password', you can use <i>.find_element_by_id('username')</i> based on the following page code. 
<img src = 'up.PNG' width = 500>
More methods can be found in <a href = 'https://selenium-python.readthedocs.io/locating-elements.html#locating-elements'>Locating Elements</a> chapter.
7. Send keys to elements, which is similar to entering keys using the keyboard. Special keys(ENTER, ALT, SHIFT,etc) can be passed by <i><b>Keys</i></b> module.

In [ ]:
#open chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
#2. create a driver instance
driver = webdriver.Chrome('C:/Users/tianyi.fang/Documents/chromedriver/chromedriver.exe', options=chrome_options)
# navigate the page
driver.get("https://infinedi.net/")

In [ ]:
#find the 'Sign in' div and do click 
driver.find_element_by_link_text('Sign In').click()

In [ ]:
print(len(driver.find_elements_by_id('username')))

In [ ]:
#find the signin section to pop up login page, the section is a link
# find the pass username and password
elem = driver.find_element_by_id('username')
elem.click()
elem.send_keys('migtest')
elem = driver.find_element_by_id('password')
elem.click()
elem.send_keys('Kverbo1430!')
#pass ENTER commend
elem.send_keys(Keys.RETURN)               

In [ ]:
driver.find_element_by_link_text('Sign In').click()

In [ ]:
driver.quit()

Seletium just open the page in a browser, Now using BrowserMob_Proxy to capture the network calls.

# Automated Tester - BrowserMob_Proxy

Antomatically get the har file after each click.<BR>
This step requires <a href = 'https://browsermob-proxy-py.readthedocs.io/en/stable/'>BrowserMob Proxy</a> package, together with seleuim package. In first time, we need to download this app from <a href = 'https://bmp.lightbody.net/'>Download BrowserMob Proxy</a> and then paste the path to server. 
- **Browsermob_proxy** allows us to manipulate HTTP requests and responses, capture HTTP content and export performnace data as a HAR file. It also allows to manipulate broswer behavior and traffic, such as stimulating network traffic, rewriting HTTP requests and responses. 
- **HAR** file is HTTP Archive, a JSON format used for tracking information between a web browser and a website. It id primarily used for identifying **1) performance issue** such as bottlenecks and slow load times and **2)page rendering problems**. Detailed information of HAR file, go <a href = 'https://developers.google.com/web/tools/chrome-devtools/network/resource-loading?utm_campaign=2016q3&utm_medium=redirect&utm_source=dcc#resource-network-timing'>this link</a>.
- **WebDriver** is a tool for automating web application testing.<br>

Steps:
1. Setup the server
2. setup driver
3. get HAR file
4. quit driver and stop server

<mark><B>Notice BroswerMob-Proxy hasn't been updated since 2016, it will lose some information of actual har file from manual download from Chrome DevTool-Network.</b><br>The alternative is browserup-proxy<br></mark>

In [ ]:
#1. Start the server
#use the last line of help('browsermobproxy') to get the module location
server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browsermob-proxy")
server.start()
proxy = server.create_proxy()
time.sleep(1)   #This helps in getting rid of some intermittent issues which can be faced dur to server taking time to start
proxy.new_har('test1') #start a new proxy, named test1

In [ ]:
#2. setup selectium driver to run on proxy
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(options = chrome_options)
driver.get("https://www.infinedi.net/")

In [ ]:
#3. Export HAR file

with open('test.har', 'w') as har_file:
    json.dump(proxy.har, har_file)

In [ ]:
#stop the server
server.stop()

Combine seletium actions and authentication to hit the Infinedi home page.

In [ ]:
#1.set up proxy server
server = Server("C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browserup-proxy")
server.start()
proxy = server.create_proxy()

#2.set up chrome driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(options = chrome_options)
#3.
proxy.new_har('mig_land')
driver.get("https://www.infinedi.net/")

driver.find_element_by_link_text('Sign In').click()  #find sign-in bottom

In [ ]:
time.sleep(10)
elem = driver.find_element_by_id('username')
elem.click()
elem.send_keys('migtest')
elem = driver.find_element_by_id('password')
elem.click()
elem.send_keys('Kverbo1430!')
#pass ENTER commend
elem.send_keys(Keys.RETURN) 

In [ ]:
proxy.new_har('mig_land_cp')
driver.get('https://www.infinedi.net/Dashboard#/claimsprocessing')
driver.refresh()
#save to local
with open('test3.har', 'w') as har_file: json.dump(proxy.har, har_file)

In [ ]:
server.stop()
driver.quit()

## Auto HAR generator w Client 
This function requires exact Username and Password for each Client. **Be careful the test username and password might be expired**.

In [ ]:
#combine as a function
def gethar(user_name, password,infinedi_dict):
    server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browserup-proxy")
    server.start()
    proxy = server.create_proxy()
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("window-size=1400,800")
    chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
    driver = webdriver.Chrome(options = chrome_options)
    driver.get("https://www.infinedi.net/")
    driver.find_element_by_link_text('Sign In').click()
    time.sleep(5)
    driver.find_element_by_id('username').send_keys(user_name) #find username blank
    driver.find_element_by_id('password').send_keys(password)
    driver.find_element_by_class_name('login-button').click()
    time.sleep(60)
    print('Logined Success')
    for name, url in infinedi_dict.items():
        file_name = name +'_' + str(datetime.now().strftime("%y%m%d%H%M%S"))
        #print(file_name)
        proxy.new_har(url)
        driver.get(url)
        driver.refresh()
        file_har = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/' +file_name+'.har'
        with open(file_har, 'w') as har_file: json.dump(proxy.har, har_file)
        time.sleep(60)
        print('Har file of'+name+ ' Done')
    #infinedi Analytics
    
    server.stop()
    driver.quit()

In [ ]:
str(datetime.now().strftime("%y%m%d%H%M%S"))

In [ ]:
mig_user = 'migtest'
mig_pass = "Kverbo1430!"
infinedi_dict = {'ClaimProcessing' : 'https://www.infinedi.net/Dashboard#/claimsprocessing/',
                'InputFileProcessing' : 'https://www.infinedi.net/Dashboard#/inputfileprocessing',
                'ERA' : 'https://www.infinedi.net/Dashboard#/erapage',
                 'UserManagement': 'https://www.infinedi.net/Dashboard#/usermanagement',
                 'News': 'https://www.infinedi.net/Dashboard#/Utilities/News',
                 'Downloads': 'https://www.infinedi.net/Dashboard#/download',
                 'Alerts': 'https://www.infinedi.net/Dashboard#/Utilities/Alerts' 
                 
                }

One time Execution

In [ ]:
gethar(user_name, password, infinedi_dict)

Automatic execution

In [ ]:
#execute this function 100 times for every 5 minutes. 
for _ in range(100):
    gethar(user_name, password, infinedi_dict)
    time.sleep(300)

add additional actions for each webpage

In [ ]:
#combine as a function
def getDashBoardhar(user_name, password,infinedi_dict):
    server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browserup-proxy")
    server.start()
    proxy = server.create_proxy()
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("window-size=1400,800")
    chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
    driver = webdriver.Chrome(options = chrome_options)
    driver.get("https://www.infinedi.net/")
    driver.find_element_by_link_text('Sign In').click()
    time.sleep(5)
    driver.find_element_by_id('username').send_keys(user_name) #find username blank
    driver.find_element_by_id('password').send_keys(password)
    driver.find_element_by_class_name('login-button').click()
    time.sleep(60)
    print('Logined Success')
    for name, url in infinedi_dict.items():
        file_name = name +'_' + str(datetime.now().strftime("%y%m%d%H%M%S"))
        #print(file_name)
        proxy.new_har(url)
        driver.get(url)
        driver.refresh()
        file_har = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/' +file_name+'.har'
        with open(file_har, 'w') as har_file: json.dump(proxy.har, har_file)
        time.sleep(60)
        print('Har file of'+name+ ' Done')
    #infinedi Analytics
    
    server.stop()
    driver.quit()

## Generate for Infinedi Analytics

In [2]:
def getanalyticshar(user_name, password):
        
    server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browserup-proxy")
    server.start()
    proxy = server.create_proxy()
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("window-size=1400,800")
    chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
    driver = webdriver.Chrome(options = chrome_options)
    
    proxy.new_har('Analytics')
    driver.get("https://analyticsmobile.infinedi.net/InfinediLogin.aspx")
    time.sleep(1)
    driver.find_element_by_id('Username').send_keys(user_name) #find username blank
    driver.find_element_by_id('password').send_keys(password)
    driver.find_element_by_id('btnSubmit').click()
    print('Logined Success')
    time.sleep(1)
    
    #select client
    select_client = Select(driver.find_element_by_id('selChildCliCodes'))
    select_client.select_by_value('MIG - MISRA CHIROPRACTIC CLINIC')
    time.sleep(1)
    select_provider = Select(driver.find_element_by_id('providerdetid'))
    select_provider.select_by_value('MISRA DC BIBHU - NPI: 1871593327')
    time.sleep(1)
    print('Select Client&provider Success')
    #print('Before click',  datetime.now())
    driver.find_element_by_id('btnSubmit').click()
    #print('After click', datetime.now())
    print('Laoding Dashboard Success')
    time.sleep(60)
    #print('After sleep',  datetime.now())
    file_name = 'Analytics_' + str(datetime.now().strftime("%y%m%d%H%M%S"))
    file_har = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/' +file_name+'.har'
    with open(file_har, 'w') as har_file: json.dump(proxy.har, har_file)
    time.sleep(10)
    print('Har file of Analytics Done')
    
    server.stop()
    driver.quit()

In [3]:
def getanalyticslocalhar(user_name, password):
        
    server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browserup-proxy-1.1.0\\bin\\browserup-proxy")
    server.start()
    proxy = server.create_proxy()
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("window-size=1400,800")
    chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
    driver = webdriver.Chrome(options = chrome_options)
    
    proxy.new_har('Analytics_local')
    driver.get("http://analyticsmobile.infinedi.local/InfinediLogin.aspx")
    time.sleep(1)
    driver.find_element_by_id('Username').send_keys(user_name) #find username blank
    driver.find_element_by_id('password').send_keys(password)
    driver.find_element_by_id('btnSubmit').click()
    print('Logined Success')
    time.sleep(1)
    
    #select client
    select_client = Select(driver.find_element_by_id('selChildCliCodes'))
    select_client.select_by_value('MIG - MISRA CHIROPRACTIC CLINIC')
    time.sleep(1)
    select_provider = Select(driver.find_element_by_id('providerdetid'))
    select_provider.select_by_value('BIBHU MISRA DC - NPI: 1871593327')
    time.sleep(1)
    print('Select Client&provider Success')
    #print('Before click', datetime.now())
    driver.find_element_by_id('btnSubmit').click()
    #print('After click', datetime.now())
    print('Laoding Dashboard Success')
    time.sleep(100)
    #print('After sleep', datetime.now())
    #phase2
    driver.find_element_by_name('ErrorAnalysisTab').click()
    time.sleep(30)
    driver.find_element_by_name('PayerAnalysisTab').click()
    time.sleep(20)
    file_name = 'Analyticslocal_' + str(datetime.now().strftime("%y%m%d%H%M%S"))
    file_har = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/' +file_name+'.har'
    with open(file_har, 'w') as har_file: json.dump(proxy.har, har_file)
    print('Har file of Analytics_local Done')
    
    server.stop()
    driver.quit()

In [4]:
mig_user = 'migtest'
mig_pass = "Kverbo1430!"
wa_prod = "https://analyticsmobile.infinedi.net/InfinediLogin.aspx"
wa_local = "http://analyticsmobile.infinedi.local/InfinediLogin.aspx"

In [ ]:
getanalyticshar(mig_user, mig_pass)

In [12]:
#for Production
for i in range(20):
    print(i)
    getanalyticshar(mig_user, mig_pass)
    time.sleep(30)

0
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
1
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
2
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
3
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
4
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
5
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
6
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
7
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
8
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har file of Analytics Done
9
Logined Success
Select Client&provider Success
Laoding Dashboard Success
Har fil

In [ ]:
print(datetime.now())

In [ ]:
getanalyticslocalhar(mig_user, mig_pass)

In [ ]:
#for local
for i in range(15): 
    getanalyticslocalhar(mig_user, mig_pass)
    time.sleep(30)

Try dynamic waiting by driver.execute_script()

In [ ]:
server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browsermob-proxy-2.1.4\\bin\\browsermob-proxy")
server.start()
proxy = server.create_proxy()
    
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1400,800")
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(options = chrome_options)
wait = WebDriverWait(driver, 5)    
proxy.new_har('Analytics')
driver.get("https://analyticsmobile.infinedi.net/InfinediLogin.aspx")
time.sleep(0.5)
driver.find_element_by_id('Username').send_keys(user_name) #find username blank
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_id('btnSubmit').click()
print('Logined Success')
time.sleep(1)
    
#select client
select_client = Select(driver.find_element_by_id('selChildCliCodes'))
select_client.select_by_value('MIG - MISRA CHIROPRACTIC CLINIC')
time.sleep(1)
select_provider = Select(driver.find_element_by_id('providerdetid'))
select_provider.select_by_value('MISRA DC BIBHU - NPI: 1871593327')
time.sleep(1)
print('Select Client&provider Success')
driver.find_element_by_id('btnSubmit').click()
wait.until(lambda driver: driver.execute_script('''return $('window.performance.timing.loadEventEnd')'''))
print('Laoding Dashboard Success')


## Auto HAR generater -- Master Username
This login authentication is Infinedi Master, which means another 'Client Selection' step is needed in the landing dashboard page.<br>
**Be careful about the selected clients that have sub clients.**

In [ ]:
ms_user = 'Tianyi.Fang'
ms_pass = 'Seattle405'

In [ ]:
#.set up proxy server
server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browsermob-proxy-2.1.4\\bin\\browsermob-proxy")
server.start()
proxy = server.create_proxy()
#set up chrome
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(options = chrome_options)
driver.get("https://www.infinedi.net/")
#login
driver.find_element_by_link_text('Sign In').click()
time.sleep(5)
driver.find_element_by_id('username').send_keys(ms_user) #find username blank
driver.find_element_by_id('password').send_keys(ms_pass)
driver.find_element_by_class_name('login-button').click()
time.sleep(10)
print('Logined Success')

In [ ]:
#select client
select_client = Select(driver.find_element_by_id('clientcodesdropdown'))
select_client.select_by_value('VAF')
driver.find_element_by_xpath('//img[contains(@src, "/Images/Apply_Button.gif")]').click()
time.sleep(10)

In [ ]:
#generate har file for each panel on the initial dashboard
for name, url in infinedi_dict.items():
    print(name, '\n', url)
#     file_name = name + str(datetime.now().strftime("%d/%m/%Y")) +'_'+ str(datetime.now().hour)
#     proxy.new_bar(file_name)
#     driver.get()

## Get TACHC har file

In [ ]:
tachc_user = 'captain'
tachc_pass = 'Qapp219$#'

### execute_script()
If you want to get Chrome DevTool Console: do driver.execute_script(). The unit is millisecond.<br>
**Performance.Timing**<br>
- navigationStart - This attribute returns the time spent after the user agent completes unloading the pervious page/document. If there was no document prior to loading the new page, navigationStart = fetchStart
- responseStart - This attribute returns the time as soon as the user-agent receives the first byte from the server or from the local sources/application cache.
- domComplete - This attribute returns the time just before the current document/page readiness is set to 'complete'. document.readyState status as 'complete' indicates that the parsing of page/document is complete& all the resources required for the page are downloaded. 
-
-
-
-
-
**WebDriverWait**<br>
WebDriverWait from **selenium** alllows page to wait until some condition. You can search for the persence of an element in the page to indicate whether the page is loaded or not. 


In [ ]:
#EXECUTE_SCRIPT
server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browsermob-proxy-2.1.4\\bin\\browsermob-proxy")
server.start()
proxy = server.create_proxy()
    
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("window-size=1400,800")
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(options = chrome_options)
    
proxy.new_har('Tachc')
driver.get("http://hccndemo.qvhsystems.com/")
driver.find_element_by_link_text('Login').click()
time.sleep(1)
driver.find_element_by_id('Username').send_keys(tachc_user) #find username blank
time.sleep(1)
driver.find_element_by_id('Password').send_keys(tachc_pass )
time.sleep(1)
driver.find_element_by_xpath("//input[@type = 'submit']").click()
nev_start = driver.execute_script("return window.performance.timing.navigationStart") #
rsp_start = driver.execute_script("return window.performance.timing.responseStart")
domComplete = driver.execute_script('return window.performance.timing.domComplete')
loadeventEnd = driver.execute_script("return window.performance.timing.loadEventEnd")
print('BackEnd:{}'.format(rsp_start-nev_start))
print('FrontEnd:{}'.format(domComplete-rsp_start))


Time for the window.console is 13 digits Unix timestamp, divided by 1000 and convert to readable time format, but since we want the time_delta, no need to convert. 

In [ ]:
print(rsp_start/1000 -nev_start/1000)

In [ ]:
def getTACHChar(user_name, password):
    server = Server(path="C:\\Users\\tianyi.fang\\Documents\\browsermob-proxy-2.1.4\\bin\\browsermob-proxy")
    server.start()
    proxy = server.create_proxy()
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("window-size=1400,800")
    chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
    driver = webdriver.Chrome(options = chrome_options)
    
    proxy.new_har('Tachc')
    driver.get("http://hccndemo.qvhsystems.com/")
    driver.find_element_by_link_text('Login').click()
    time.sleep(1)
    driver.find_element_by_id('Username').send_keys(user_name) #find username blank
    time.sleep(1)
    driver.find_element_by_id('Password').send_keys(password)
    time.sleep(1)
    driver.find_element_by_xpath("//input[@type = 'submit']").click()
    time.sleep(180)
    print('Logined Success')
    file_name = 'Tachc_' + str(datetime.now().strftime("%y%m%d%H%M%S"))
        #print(file_name)
    
    #driver.refresh()
    file_har = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/' +file_name+'.har'
    with open(file_har, 'w') as har_file: json.dump(proxy.har, har_file)
    time.sleep(10)
    print('Har file Done')
    #infinedi Analytics
    
    server.stop()
    driver.quit()

In [ ]:
getTACHChar(tachc_user, tachc_pass)

# Get performance
<a href = 'https://github.com/browserup/browserup-proxy-py'>Here</a> is a new version of Proxy for auto scrubbing har file.

In [ ]:
# #enable browser logging
d = DesiredCapabilities.CHROME
d['goog:loggingPrefs'] = {'performance': 'ALL'}
driver = webdriver.Chrome(desired_capabilities=d)
driver.get("https://www.infinedi.net/#")
driver.find_element_by_link_text('Sign In').click()
time.sleep(5)
driver.find_element_by_id('username').send_keys(mig_user) #find username blank
driver.find_element_by_id('password').send_keys(mig_pass)
driver.find_element_by_class_name('login-button').click()
with open('performance.json', 'w') as per_json: json.dump(driver.get_log('performance'), per_json)
#
# with open('performance.csv', 'w', newline = '') as f:
#     writer = csv.writer(f)
    
#     for entry in driver.get_log('performance'):
#         writer.writerows()
    

# Convert HAR file to CSV
Follow guide in <a href = 'https://gist.github.com/tomatohater/8853161'>parsehar.py</a> to convert har file into csv

In [ ]:
def har2csv(harfile_path, csv_path):
    #read a har from path and dump to stdout
    harfile = open(harfile_path, encoding="utf-8")
    harfile_json = json.loads(harfile.read())
    i = 0
    with open(csv_path + harfile_path[:-3] + 'csv', 'w', newline='') as f:
        csv_file = csv.writer(f)
        csv_file.writerow(['id', 'url', 'hostname','StartTime', 'Total_time','header_size', 'rsp_status', 'rsp_body_size(bytes)', 'rsp_body_size(kilobytes)', 
                           'rsp_content_mimetype', 'rsp_content_size', 'wait_time', 'send_time', 'receive_time', 'blocked_time', 'dns_time',
                          'ssl_time', 'connect_time', 'priority', 'resourcetype', 'connection','fromCache'])  #19
        # variables in "entries" 
        
        for entry in harfile_json['log']['entries']:
            i += 1
            start_time = entry["startedDateTime"]
            time = entry['time']
            url = entry['request']['url']
            urlparts = urlparse(entry['request']['url']) #hostname
            header_size = entry['response']['headersSize']
            rsp_status = entry['response']['status']
            rsp_size_bytes = entry['response']['bodySize']
            rsp_size_kilobytes = float(entry['response']['bodySize'])/1024
            
            mimetype = 'unknown'
            if 'mimeType' in entry['response']['content']:
                mimetype = entry['response']['content']['mimeType']
            content_size = entry['response']['content']['size']
                        
            send = entry['timings']['send']
            wait = entry['timings']['wait']
            receive = entry['timings']['receive']
            blocked =  entry['timings']['blocked']
            dns = entry['timings']['dns']
            ssl = entry['timings']['ssl']
            connect = entry['timings']['connect']
            
            #some har does not have the following measures, if Null, leave Null, otherwise, convert.
            priority, resourcetype, connection, fromCache = 'NaN','NaN','NaN','NaN'
            if "_priority" in entry:
                priority = entry['_priority']
            if "_resourceType" in entry:
                resourcetype = entry["_resourceType"]
            if '_fromCache' in entry:
                fromCache = entry['_fromCache']
            if "connection" in entry:
                connection = entry["connection"]

            csv_file.writerow([i, url, urlparts.hostname,start_time, time,header_size, rsp_status, rsp_size_bytes, rsp_size_kilobytes, mimetype,content_size,
                              wait, send, receive, blocked, dns, ssl, connect, priority, resourcetype, connection, fromCache])

In [ ]:
har2csv('Tachc_191210120515.har', csv_path)

In [6]:
def har2csv_total(csv_path, cat_type, har_path, write_type):
        
    with open(csv_path + cat_type+'_total.csv', write_type, newline='') as f:
        csv_file = csv.writer(f)
        csv_file.writerow(['id', 'method','url', 'hostname','StartTime', 'Total_time','header_size', 'rsp_status', 
                           'rsp_body_size(bytes)', 'rsp_body_size(kilobytes)', 'rsp_content_mimetype', 
                           'wait_time', 'send_time', 'receive_time', 'blocked_time', 'dns_time','ssl_time', 'connect_time', 
                           'category'])  #19
        # variables in "entries" 
        for file in glob.glob(cat_type+'_'+'*[0-9].har'):
            print(file)
            #read a har from path and dump to stdout
            harfile = open(file, encoding="utf-8")
            harfile_json = json.loads(harfile.read())
            i = 0
            
            for entry in harfile_json['log']['entries']:
                i += 1
                start_time = entry["startedDateTime"]
                time = entry['time']
                method = entry['request']['method']
                url = entry['request']['url']
                urlparts = urlparse(entry['request']['url']) #hostname
                header_size = entry['response']['headersSize']
                rsp_status = entry['response']['status']
                rsp_size_bytes = entry['response']['bodySize']
                rsp_size_kilobytes = float(entry['response']['bodySize'])/1024
            
                mimetype = 'unknown'
                if 'mimeType' in entry['response']['content']:
                    mimetype = entry['response']['content']['mimeType']
                        
                send = entry['timings']['send']
                wait = entry['timings']['wait']
                receive = entry['timings']['receive']
                blocked =  entry['timings']['blocked']
                dns = entry['timings']['dns']
                ssl = entry['timings']['ssl']
                connect = entry['timings']['connect']

                csv_file.writerow([i,method, url, urlparts.hostname,start_time, time,header_size, rsp_status, rsp_size_bytes, rsp_size_kilobytes, mimetype,
                              wait, send, receive, blocked, dns, ssl, connect,cat_type])
            harfile.close()
            
            shutil.copy2(file, har_path)
            os.remove(file)
    

In [7]:
csv_path = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/CSVs/'
csv_total_path = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/CSVs_total/'
har_path = 'C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/HARs/'          

Convert multiple HAR files into 1 csv

In [13]:
har2csv_total(csv_total_path, 'Analytics', har_path, 'a')

Analytics_191219075955.har
Analytics_191219080157.har
Analytics_191219080356.har
Analytics_191219080552.har
Analytics_191219080750.har
Analytics_191219080948.har
Analytics_191219081146.har
Analytics_191219081343.har
Analytics_191219081542.har
Analytics_191219081739.har
Analytics_191219081936.har
Analytics_191219082133.har
Analytics_191219082329.har
Analytics_191219082527.har
Analytics_191219082730.har
Analytics_191219082930.har
Analytics_191219083133.har
Analytics_191219083329.har
Analytics_191219083527.har
Analytics_191219083724.har
Analytics_191219084550.har
Analytics_191219084751.har
Analytics_191219084949.har
Analytics_191219085146.har
Analytics_191219085342.har
Analytics_191219085542.har
Analytics_191219085740.har
Analytics_191219085942.har
Analytics_191219090142.har
Analytics_191219090345.har
Analytics_191219090553.har
Analytics_191219090750.har
Analytics_191219090951.har
Analytics_191219091153.har
Analytics_191219091355.har
Analytics_191219091556.har
Analytics_191219091753.har
A

In [ ]:
har2csv_total(csv_total_path, 'Analyticslocal', har_path, 'w')

In [ ]:
os.listdir()

Move all HAR files to HARs folder

In [ ]:
<img src = "C:/Users/tianyi.fang/Desktop/TF/projects/WebPage_Audit/Chorme_Audit/webpage_timing_overview.png">

# Argparse the har file
This module is the 'recommended command-line parsing module in the Python standard library', which let you provide values for variables at runtime. It is useful when you want to change values for some variables, allow 'input' and 'output' values, etc.
Check <a href = 'https://docs.python.org/3/library/argparse.html'>here</a> for tutorial.
# Some research
<img src = "webpage_timing-overview.png" height = 900, width = 700>


In [ ]:
#Simple example
ex_parser = argparse.ArgumentParser(description='Short Example')
ex_parser.add_argument('-a', action = 'store_true', default = False) #a boolean option
ex_parser.add_argument('-b', action = 'store', dest = 'b') # a string option
ex_parser.add_argument('-c', action = 'store', dest = 'c', type = int) # an integer option
ex_arg = ex_parser.parse_args(['-a', '-bval', '-c', '3'])
print(ex_arg)
print(ex_arg.c)

In [ ]:
parser = argparse.ArgumentParser(prog = 'parsehar', description='Parse .har files into comma separated values (csv).')
parser.add_argument('harfile', type = str, help = har_path)
args = parser.parse_args(har_path.split())
print(args.harfile[0])

## Some notes about HAR files
Once you have the HAR files, you can convert to csv files or visualize it. 
1.**What to look for HAR files**<br>
- How many items are loading. Typical items are javascript files, css files, templates and images.